In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp, col, lit

source_df = spark.table("bronze_db.customers")

# First run (table does not exist)
if not spark.catalog.tableExists("silver_db.customers"):
    silver_df = (
        source_df
        .withColumn("created_ts", current_timestamp())
        .withColumn("modified_ts", current_timestamp())
    )

    silver_df.write.format("delta").saveAsTable("silver_db.customers")

else:
    target = DeltaTable.forName(spark, "silver_db.customers")

    # Prepare source with ETL timestamps
    staged_df = (
        source_df
        .withColumn("etl_ts", current_timestamp())
    )

    (
        target.alias("t")
        .merge(
            staged_df.alias("s"),
            "t.customer_id = s.customer_id"
        )
        .whenMatchedUpdate(
    condition="""
        NOT (
            t.name <=> s.name AND
            t.email <=> s.email AND
            t.city <=> s.city
        )
    """,
    set={
        "name": "s.name",
        "email": "s.email",
        "city": "s.city",
        "modified_ts": "s.etl_ts"
    }
)

        .whenNotMatchedInsert(values={
            "customer_id": "s.customer_id",
            "name": "s.name",
            "email": "s.email",
            "city": "s.city",
            "created_ts": "s.etl_ts",
            "modified_ts": "s.etl_ts"
        })
        .execute()
    )
